# Import packages

In [1]:
import os, re, math

# Load file

In [2]:
filename = os.path.join("..", "..", "CDS-LANG", "100_english_novels", "corpus", "Forster_Howards_1910.txt")

In [3]:
with open(filename, "r") as f:
    text = f.read()

# Set keyword and window size

In [4]:
keyword = "love"
window_size = 5

# Tokenizer using ```regex```

In [5]:
# A quick regex tokenizer for splitting files below
def tokenize(input_string):
    tokenizer = re.compile(r"\W+")
    return tokenizer.split(input_string)

# Tokenize text and remove punctuation

In [6]:
# ... tokenize the text and remove punctuation
tokenized_text = []

for word in tokenize(text):
    # Lowercase
    lowercase = word.lower()
    # cleanup punctuation etc
    cleaned = re.sub(r'[^\w\s]', '', lowercase)
    tokenized_text.append(cleaned)

# Get context words

In [7]:
# create temporary list
tmp = []
# For the target word... 
for idx,word in enumerate(tokenized_text):
    # If it's the keyword...
    if word == keyword:
        # Left context catch start of list
        left_context = max(0, idx-window_size)
        right_context = idx+window_size
        # ... extract all words ± 5 and add to tmp list.
        full_context = tokenized_text[left_context:idx] + tokenized_text[idx+1:right_context]
        # append to list
        tmp.append(full_context)

# Flatten all of the lists

In [8]:
# Flatten list
flattened_list = []
# For each sublist in list of lists
for sublist in tmp:
    # For each item in sublist
    for item in sublist:
        # append
        flattened_list.append(item)

# Count how many times each collocate occurs

In [9]:
# Create a list of collocate counts
collocate_counts = []

# for every collocate 
for word in set(flattened_list):
    # Count how often each word appears as a collocate
    count = flattened_list.count(word)
    # Append tuple of word and count to list
    collocate_counts.append((word, count))

# Define MI function

```MI = log ( (AB * sizeCorpus) / (A * B * span) ) / log (2)```

- A = frequency of node word
- B = frequency of collocate
- AB = frequency of collocate near the node word
- sizeCorpus= size of corpus
- span = span of words
- log (2) is literally the log10 of the number 2

In [29]:
def MI(A, B, AB, span, corpus_size):
    score = math.log((AB * corpus_size) / (A * B * span)) / math.log(2)
    return score

# Calculate MI for every context word

In [33]:
keyword_freq = tokenized_text.count(keyword)
corpus_size = len(tokenized_text)
outfile = "collocates.csv"


out_list = []
for tup in collocate_counts:
    coll_text = tup[0]
    coll_count = tup[1]
    total_occurrences = tokenized_text.count(coll_text)
    score = MI(keyword_freq, coll_count, total_occurrences, 10, corpus_size)
    out_list.append[(coll_text, coll_count, total_occurrences, score)]

    
header = ["text", "collocate_count", "total_count", "MI"] 
with open(outfile, 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    writer.writerow(data)

TypeError: TextIOWrapper.write() takes exactly one argument (3 given)